In [ ]:
from IPython.display import Audio


In [ ]:
import sys
import glob
import importlib
sys.path.append('/home/jupyter/rfcx_submission/scripts')

In [ ]:
import librosa
import librosa.display

In [ ]:
import load_mel
from augment import do_aug

In [ ]:
def show_audio(y,sr):
    librosa.display.waveplot(y, sr=sr)

    spec= load_mel.get_spectrogram(y,sr,apply_denoise=False,return_audio=False)
    load_mel.plot_feature(spec)
    
    return Audio(y,rate=sr)

In [ ]:
audio_files= glob.glob('/home/jupyter/rfcx/data/*/*.flac')
# audio_files= glob.glob('/home/jupyter/librispeech/LibriSpeech/test-other/1688/142285/*.flac')
len(audio_files)


In [ ]:
input_file , input_file2 = audio_files[1] , audio_files[2]
input_file

In [ ]:
SAMPLE_RATE = 16000


In [ ]:
y,sr= load_mel.load_audio(input_file, SAMPLE_RATE)
show_audio(y, SAMPLE_RATE)

In [ ]:
y_deniosed= load_mel.denoise(y, sr)
show_audio(y_deniosed,sr)

In [ ]:
y_aug= do_aug(y_deniosed, sr)
show_audio(y_aug,sr)

In [ ]:
y2,sr= load_mel.load_audio(input_file2, SAMPLE_RATE)
show_audio(y2, SAMPLE_RATE)

In [ ]:
y_deniosed2= load_mel.denoise(y2, sr)
show_audio(y_deniosed2,sr)

In [ ]:
mixed_up= y+y2
show_audio(mixed_up,sr)

In [ ]:
mixed_up_denoised= y_deniosed+ y_deniosed2
show_audio(mixed_up_denoised,sr)

In [ ]:
mixed_up_denoised= y+ y_deniosed2
show_audio(mixed_up_denoised,sr)

In [ ]:
mixed_up_denoised= y_deniosed+ y2
show_audio(mixed_up_denoised,sr)

In [ ]:
alpha= 0.5
mixed_up_denoised= alpha*y_deniosed+ (1-alpha)*y_deniosed2
show_audio(mixed_up_denoised,sr)

In [ ]:
import numpy as np
import random

In [ ]:
class MixUp:
    def __init__(self ,denoise,sr):
        '''
        Args:
            denoise : function object to denoise input signals
        '''
        self.denoise = denoise
        self.sr= sr

    def __call__(self, y1, y2, alpha=None):
        if not alpha :
            alpha= np.random.uniform(0.3, 0.7)
        
        option = random.choice([0,1,2,3])
        
        a,b= y1.copy() , y2.copy()
        
        if option==1:
            a= self.denoise(a, sr)
        elif option==2:
            b= self.denoise(b, sr)
        elif option==3:
            a= self.denoise(a, sr)
            b= self.denoise(b, sr)
        else:
            #option==0
            pass
        
        y= alpha*a +(1-alpha)*b
        print(option)
        return alpha,y

In [ ]:
mixup = MixUp(load_mel.denoise, SAMPLE_RATE)

In [ ]:
alpha,y3 = mixup(y, y2 )
print(alpha)
show_audio(y3,SAMPLE_RATE)
